In [1]:
%load_ext watermark
%watermark -a 'cs224' -u -d -v -p numpy,pandas,matplotlib,sklearn,h5py,zipline

cs224 
last updated: 2020-12-29 

CPython 3.6.12
IPython 7.16.1

numpy 1.19.1
pandas 0.22.0
matplotlib 3.3.1
sklearn 0.23.2
h5py 2.10.0
zipline 1.4.1


In [2]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import sklearn, sklearn.pipeline, sklearn.model_selection, sklearn.preprocessing, sklearn.linear_model
import re, fnmatch

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(linewidth=1000)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

SEED = 42
np.random.seed(SEED)

sns.set()

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [4]:
import os,sys
path = os.path.realpath(os.path.abspath('') + '/csi_futures_data')
sys.path.append(path)

In [5]:
%load_ext autoreload
%autoreload 1
%aimport csi_futures_data

In [6]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

def display_graphs_side_by_side(*args):
    html_str='<table><tr>'
    for g in args:
        html_str += '<td>'
        html_str += g._repr_svg_()
        html_str += '</td>'
    html_str += '</tr></table>'
    display_html(html_str,raw=True)
    

display(HTML("<style>.container { width:70% !important; }</style>"))

In [7]:
# the following list is a hand curated list of futures as a combination of data from:
# - Following the Trend
# - Trading Evolved
# - zipline.finance.constants.ROOT_SYMBOL_TO_ETA
# - zipline.finance.constants.FUTURE_EXCHANGE_FEES_BY_SYMBOL
csi_futures_data_df = csi_futures_data.futures_lookup_.copy()
csi_futures_data_df

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector
0,KC2,KC,37500.0,0.01,Coffee,CSCE,Agricultural
1,C2,_C,5000.0,0.01,Corn,CBT,Agricultural
2,C2,CN,5000.0,0.01,Corn,CBT,Agricultural
3,CT2,CT,50000.0,0.01,Cotton #2,NYCE,Agricultural
4,LH,LH,40000.0,0.01,Lean Hogs,CME,Agricultural
5,LC,LC,40000.0,0.01,Live Cattle,CME,Agricultural
6,LB,LB,110.0,1.00,Lumber,CME,Agricultural
7,O2,_O,5000.0,0.01,Oats-CBT,CBT,Agricultural
8,O2,OA,5000.0,0.01,Oats-CBT,CBT,Agricultural
9,OJ2,OJ,15000.0,0.01,Orange Juice,NYCE,Agricultural


In [8]:
# the meta.csv is from the package that Andreas Clenow provides for his random_futures_data: https://www.dropbox.com/s/etocgt9zgeedo22/data.zip?dl=0
futures_meta_df = pd.read_csv('meta.csv', index_col=0)
futures_meta_df

,root_symbol,multiplier,minor_fx_adj,description,exchange,sector
0,AD,100000,1.00,AUD/USD,CME,Currency
1,BO,600,0.01,Soybean Oil,CBT,Agricultural
2,BP,62500,1.00,GBP/USD,CME,Currency
3,_C,5000,0.01,Corn,CBT,Agricultural
4,CC,10,1.00,Cocoa,NYCE,Agricultural
5,CD,100000,1.00,CAD/USD,CME,Currency
6,CL,1000,1.00,Crude Oil,NYMEX,Non-Agricultural
7,CT,50000,0.01,Cotton #2,NYCE,Agricultural
8,CU,125000,1.00,EUR/USD,CME,Currency
9,DA,200000,1.00,Class III Milk,CME,Agricultural


In [9]:
# outdated: http://www.csidata.com/factsheets/factsheet-futures.html
# csidata_futures_fact_sheet_df = pd.read_excel('~/2020-10-13-csidata-futures-fact-sheet.xlsx')
# csidata_futures_fact_sheet_df.drop(csidata_futures_fact_sheet_df.columns[1], axis=1, inplace=True)
# csidata_futures_fact_sheet_df.head()

In [10]:
# New, up to date: http://www.csidata.com/factsheets.php?type=commodity&format=htmltable&exchangeid=
csidata_futures_fact_sheet_df = pd.read_csv('./2020-10-29-commodityfactsheet.csv', parse_dates=['StartDate', 'EndDate'])
csidata_futures_fact_sheet_df = csidata_futures_fact_sheet_df[['UACsiNumber', 'SymbolUA', 'ExchangeSymbol', 'Exchange', 'Name', 'SessionType', 'IsActive', 'TerminalPointValue', 'FullPointValue', 'Currency', 'ContractSize', 'Units', 'MinimumTick', 'TickValue', 'StartDate', 'EndDate',\
                                                               'LinkSymbol', 'ConversionFactorCode', 'HasCurrentDayVolume', 'HasCurrentDayOpenInterest', 'HasKnownExpirationDates', 'LastTotalVolume']]
csidata_futures_fact_sheet_df.head()

,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume
0,1,IB,NaN,CBT,Broilers-Iced,NaN,0,3.0,300.0,USD,30000 lbs,cents/lb,1.0,3.0,1969-08-19,1981-01-22,NaN,2,0,0,0,NaN
1,2,LC,LC,CME,Live Cattle,Combined,1,4.0,400.0,USD,40000 lbs,cents/lb,2.5,10.0,1964-11-30,2020-10-28,NaN,2,0,0,1,57781.0
2,3,CC,CC,ICEUS,Cocoa,RTH,1,10.0,10.0,USD,10 tonnes,USD/tonne,1.0,10.0,1965-12-30,2020-10-28,CC2 starts at 1965-12-30,0,0,0,1,31624.0
3,4,LH,LH,CME,Lean Hogs,Combined,1,4.0,400.0,USD,40000 lbs,cents/lb,2.5,10.0,1966-02-28,2020-10-28,NaN,2,0,0,1,30958.0
4,5,PB,PB,CME,Pork Bellies,RTH,0,4.0,400.0,USD,40000 lbs,cents/lb,2.5,10.0,1961-09-18,2011-07-15,NaN,2,0,0,0,NaN


In [11]:
s1 = set(csi_futures_data_df.root_symbol)
s2 = set(futures_meta_df.root_symbol)

In [12]:
# verify that all symbols that are present in the meta.csv are present in csi_futures_data_df
s2 - s1

set()

## Verify the multipliers and minor_fx_adj between meta.csv and what the CSI data factsheet says

My understanding is that the values in `csi_futures_data_df` are correct and the values in `meta.csv` are incorrect.

In [13]:
ldf = pd.merge(csi_futures_data_df, futures_meta_df, how='left', on=['root_symbol'])
ldf.head()

,csi_symbol,root_symbol,multiplier_x,minor_fx_adj_x,description_x,exchange_x,sector_x,multiplier_y,minor_fx_adj_y,description_y,exchange_y,sector_y
0,KC2,KC,37500.0,0.01,Coffee,CSCE,Agricultural,37500.0,0.01,Coffee Arabica,NYCE,Agricultural
1,C2,_C,5000.0,0.01,Corn,CBT,Agricultural,5000.0,0.01,Corn,CBT,Agricultural
2,C2,CN,5000.0,0.01,Corn,CBT,Agricultural,NaN,NaN,NaN,NaN,NaN
3,CT2,CT,50000.0,0.01,Cotton #2,NYCE,Agricultural,50000.0,0.01,Cotton #2,NYCE,Agricultural
4,LH,LH,40000.0,0.01,Lean Hogs,CME,Agricultural,40000.0,0.01,Lean Hogs,CME,Agricultural


In [14]:
ldf[(ldf['multiplier_x'] != ldf['multiplier_y']) & ldf['root_symbol'].isin(futures_meta_df['root_symbol'])]

,csi_symbol,root_symbol,multiplier_x,minor_fx_adj_x,description_x,exchange_x,sector_x,multiplier_y,minor_fx_adj_y,description_y,exchange_y,sector_y
21,BO2,BO,60000.0,0.01,Soybean Oil,CBT,Agricultural,600.0,0.01,Soybean Oil,CBT,Agricultural
24,SB2,SB,112000.0,0.01,Sugar #11,NYCE,Agricultural,120000.0,0.01,Sugar #11,NYCE,Agricultural
63,ER2,TF,50.0,1.00,Russel 2000 (E-mini),CME,Equities,100.0,1.00,Russel 2000 (E-mini),NYFE,Equities


In [15]:
ldf[(ldf['minor_fx_adj_x'] != ldf['minor_fx_adj_y']) & ldf['root_symbol'].isin(futures_meta_df['root_symbol'])]

,csi_symbol,root_symbol,multiplier_x,minor_fx_adj_x,description_x,exchange_x,sector_x,multiplier_y,minor_fx_adj_y,description_y,exchange_y,sector_y
18,DA,DA,200000.0,0.01,Class III Milk,CME,Agricultural,200000.0,1.00,Class III Milk,CME,Agricultural
56,MP,MP,500000.0,1.00,MEP/USD,CME,Currency,500000.0,0.01,MEP/USD,CME,Currency


In [16]:
ldf[(ldf['exchange_x'] != ldf['exchange_y']) & ldf['root_symbol'].isin(futures_meta_df['root_symbol'])]

,csi_symbol,root_symbol,multiplier_x,minor_fx_adj_x,description_x,exchange_x,sector_x,multiplier_y,minor_fx_adj_y,description_y,exchange_y,sector_y
0,KC2,KC,37500.0,0.01,Coffee,CSCE,Agricultural,37500.0,0.01,Coffee Arabica,NYCE,Agricultural
63,ER2,TF,50.0,1.00,Russel 2000 (E-mini),CME,Equities,100.0,1.00,Russel 2000 (E-mini),NYFE,Equities


## Verify the FullPointValue as given by CSI data's factsheet with the value in csi_futures_data_df

In [17]:
ldf_ = pd.merge(csi_futures_data_df, csidata_futures_fact_sheet_df, how='left', left_on=['csi_symbol'], right_on=['SymbolUA'])
#ldf_ = ldf_[['csi_symbol', 'root_symbol', 'multiplier', 'minor_fx_adj', 'description', 'exchange', 'sector', 'EXCHANGE', 'EXCHANGE SYMBOL', 'NAME', 'ACTIVE', 'CONTRACT SIZE', 'UNIT OF MEASURE', 'CURRENCY', 'FULL POINT VALUE']]
#ldf_['numeric_full_point_value'] = ldf_['FULL POINT VALUE'].str.extract(r'^(\d+).*$').astype(np.int)
ldf_['calc_poin_value'] = (ldf_['multiplier'] * ldf_['minor_fx_adj'])#.astype(np.int)
ldf_.head()

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value
0,KC2,KC,37500.0,0.01,Coffee,CSCE,Agricultural,1146,KC2,KC,ICEUS,Coffee,Combined,1,3.75,375.0,USD,37500 lbs,cents/pound,5.0,18.75,1972-08-16,2020-10-28,KC starts at 1972-08-16,2,0,0,1,31867.0,375.0
1,C2,_C,5000.0,0.01,Corn,CBT,Agricultural,412,C2,C,CBT,Corn,Combined,1,6.25,50.0,USD,5000 bu,cents/bu,2.0,12.50,1949-01-03,2020-10-28,C starts at 1946-01-02,-1,0,0,1,426691.0,50.0
2,C2,CN,5000.0,0.01,Corn,CBT,Agricultural,412,C2,C,CBT,Corn,Combined,1,6.25,50.0,USD,5000 bu,cents/bu,2.0,12.50,1949-01-03,2020-10-28,C starts at 1946-01-02,-1,0,0,1,426691.0,50.0
3,CT2,CT,50000.0,0.01,Cotton #2,NYCE,Agricultural,1144,CT2,CT,ICEUS,Cotton #2,Combined,1,5.00,500.0,USD,50000 lbs,cents/pound,1.0,5.00,1967-03-22,2020-10-28,CT starts at 1967-03-22,2,0,0,1,35749.0,500.0
4,LH,LH,40000.0,0.01,Lean Hogs,CME,Agricultural,4,LH,LH,CME,Lean Hogs,Combined,1,4.00,400.0,USD,40000 lbs,cents/lb,2.5,10.00,1966-02-28,2020-10-28,NaN,2,0,0,1,30958.0,400.0


In [18]:
# This here is the most important as this makes sure that we agree on the point value
# The expected result is that the dataframe has 0 entries
ldf_[ldf_['FullPointValue'] != ldf_['calc_poin_value']]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value


In [19]:
# In general I tried to use the symbols that represent the combined floor and electronic session
ldf_[~ldf_['SessionType'].str.contains(r'Combined').astype(np.bool)]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value


In [20]:
list(ldf_[~ldf_['SessionType'].str.contains(r'Combined').astype(np.bool)].loc[:,'csi_symbol'].values)

[]

In [21]:
# The name of the exchange is irrelevant, but just for the sake of showing the differences
ldf_[ldf_['exchange'] != ldf_['Exchange']].loc[:,['csi_symbol', 'root_symbol', 'multiplier', 'minor_fx_adj', 'description', 'sector', 'exchange', 'Exchange']]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,sector,exchange,Exchange
0,KC2,KC,37500.00,0.01,Coffee,Agricultural,CSCE,ICEUS
3,CT2,CT,50000.00,0.01,Cotton #2,Agricultural,NYCE,ICEUS
9,OJ2,OJ,15000.00,0.01,Orange Juice,Agricultural,NYCE,ICEUS
13,LSU,LS,50.00,1.00,Sugar #5(White),Agricultural,EURONEXT,LCE
20,LRC,LR,10.00,1.00,Robusta Coffee New (LCE),Agricultural,EURONEXT,LCE
23,CC2,CC,10.00,1.00,Cocoa,Agricultural,NYCE,ICEUS
24,SB2,SB,112000.00,0.01,Sugar #11,Agricultural,NYCE,ICEUS
25,XC2,CM,1000.00,0.01,Corn E-Mini,Agricultural,CBOT,CBT
26,XS2,MS,1000.00,0.01,Soybeans E-Mini,Agricultural,CBOT,CBT
28,LCO,LO,1000.00,1.00,Brent Crude,Non-Agricultural,ICE,ICEEUROPE


In [22]:
# All futures trade in USD
ldf_[ldf_['Currency'] != 'USD']

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value


In [23]:
# XXX This here is also very important as this makes sure that we agree on the minor_fx_adj
#     Only for "Class III Milk" the pattern is a bit different, but this USD/CWT should also mean that minor_fx_adj is 0.01, e.g. this should be correct.
ldf_['factsheet_minor_fx_adj'] = -1.0
selector = ldf_['Units'].str.startswith('cents').astype(np.bool)
ldf_.loc[selector, 'factsheet_minor_fx_adj'] = 0.01
ldf_.loc[~selector, 'factsheet_minor_fx_adj'] = 1.0
ldf_[ldf_['factsheet_minor_fx_adj'] != ldf_['minor_fx_adj']]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value,factsheet_minor_fx_adj
18,DA,DA,200000.0,0.01,Class III Milk,CME,Agricultural,404,DA,DA,CME,Class III Milk,Combined,1,20.0,2000.0,USD,200000 lbs.,USD/CWT,1.0,20.0,1996-01-11,2020-10-28,NaN,2,0,0,1,2095.0,2000.0,1.0


## Verify that all contracts are still traded

In [24]:
# make sure that all contracts are still trading
ldf_[ldf_['LastTotalVolume'] < 1.0]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value,factsheet_minor_fx_adj


In [25]:
# make sure that all contracts are still trading
ldf_[ldf_['EndDate'] != pd.Timestamp('2020-10-28')]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value,factsheet_minor_fx_adj


In [26]:
# not all contracts existed at the beginning of the time winow
ldf_[ldf_['StartDate'] > pd.Timestamp('2000-12-31')]

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume,calc_poin_value,factsheet_minor_fx_adj
20,LRC,LR,10.0,1.00,Robusta Coffee New (LCE),EURONEXT,Agricultural,816,LRC,RC,LCE,Robusta Coffee,NaN,1,10.000,10.0,USD,10 tonne,USD/tonne,1.0,10.000,2008-01-11,2020-10-28,LKD starts at 1991-03-01,0,1,0,1,14463.0,10.0,1.00
25,XC2,CM,1000.0,0.01,Corn E-Mini,CBOT,Agricultural,1192,XC2,YC,CBT,mini Corn,Combined,1,1.250,10.0,USD,1000 bushels,cents/bushel,1.0,1.250,2007-05-15,2020-10-28,NaN,-1,0,0,1,2589.0,10.0,0.01
26,XS2,MS,1000.0,0.01,Soybeans E-Mini,CBOT,Agricultural,1194,XS2,YK,CBT,mini Soybeans,Combined,1,1.250,10.0,USD,1000 bushels,cents/bushel,1.0,1.250,2007-05-15,2020-10-28,NaN,-1,0,0,1,3510.0,10.0,0.01
31,RB2,RB,42000.0,1.00,Gasoline-Reformulated Blendstock,NYMEX,Non-Agricultural,976,RB2,RB,NYMEX,Gasoline-Reformulated Blendstock,Combined,1,4.200,42000.0,USD,42000 gallons,USD/gallon,1.0,4.200,2005-10-03,2020-10-28,HU2 starts at 1984-12-03,4,0,0,1,126660.0,42000.0,1.00
39,ER,AI,100.0,1.00,Bloomberg Commodity Index Futures,CME,Non-Agricultural,1081,ER,AW,CBT,Bloomberg Commodity Index (DJ UBS Commodity In...,NaN,1,10.000,100.0,USD,USD 100 x Index,points,1.0,10.000,2006-10-13,2020-10-28,NaN,1,0,0,1,755.0,100.0,1.00
40,AC,ET,29000.0,1.00,Ethanol,CBT,Non-Agricultural,962,AC,ACF,CBT,Ethanol,NaN,1,29.000,29000.0,USD,29000 gallons,USD/gallon,1.0,29.000,2005-03-23,2020-10-28,NaN,3,0,0,1,14.0,29000.0,1.00
41,QG,QG,2500.0,1.00,Natural Gas E-mini,NYMEX,Non-Agricultural,724,QG,QG,NYMEX,e-mini Henry Hub Natural Gas,NaN,1,2.500,2500.0,USD,2500 mmBtu,$/mmBtu,5.0,12.500,2002-06-17,2020-10-28,NaN,3,0,0,1,8516.0,2500.0,1.00
42,QM,QM,500.0,1.00,Crude Oil E-Mini,NYMEX,Non-Agricultural,725,QM,QM,NYMEX,e-mini Crude Oil,NaN,1,0.500,500.0,USD,500 barrels,$/barrels,25.0,12.500,2002-06-17,2020-10-28,NaN,3,0,0,1,14565.0,500.0,1.00
43,IRB,XB,42000.0,1.00,RBOB Gasoline Futures,NYMEX,Non-Agricultural,772,IRB,N,ICEEUROPE,RBOB Gasoline,NaN,1,4.200,42000.0,USD,42000 gallons,USD/gallons,1.0,4.200,2006-04-24,2020-10-28,NaN,4,0,0,1,64543.0,42000.0,1.00
59,M6E,EU,12500.0,1.00,E-micro EUR/USD Futures,CME,Currency,1415,M6E,M6E,CME,e-micro EUR/USD,NaN,1,0.125,12500.0,USD,EUR 12500,USD / EUR,10.0,1.250,2009-03-23,2020-10-28,NaN,5,0,0,1,15432.0,12500.0,1.00


In [27]:
# Some indices are the continuation of others
csidata_futures_fact_sheet_df[csidata_futures_fact_sheet_df['SymbolUA'].isin(['LRC', 'LKD', 'LKR'])]

,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume
97,50,LKR,NaN,LCE,Coffee-Robusta,NaN,0,0.5,5.0,GBP,5 tonnes,GBP/tonne,1.0,0.5,1968-01-02,1992-03-31,NaN,1,0,0,1,NaN
217,148,LKD,NaN,LCE,Coffee-Robusta,NaN,0,5.0,5.0,USD,5 tonnes,USD/tonne,1.0,5.0,1991-03-01,2009-01-30,LKR starts at 1968-01-02,0,1,0,1,NaN
901,816,LRC,RC,LCE,Robusta Coffee,NaN,1,10.0,10.0,USD,10 tonne,USD/tonne,1.0,10.0,2008-01-11,2020-10-28,LKD starts at 1991-03-01,0,1,0,1,14463.0


In [28]:
# Some indices are the continuation of others
csidata_futures_fact_sheet_df[csidata_futures_fact_sheet_df['SymbolUA'].isin(['RB2', 'HU2', 'HU'])]

,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume
282,224,HU,RBOB,NYMEX,Gasoline Unleaded,RTH,0,4.2,42000.0,USD,42000 gallons,USD/gal,1.0,4.2,1984-12-03,2006-12-29,NaN,4,0,0,1,NaN
926,858,HU2,YQ,NYMEX,Unleaded Gasoline,Combined,0,4.2,42000.0,USD,42000 gallons,USD/gallon,1.0,4.2,1984-12-03,2006-12-29,HU starts at 1984-12-03,4,0,0,0,NaN
1043,976,RB2,RB,NYMEX,Gasoline-Reformulated Blendstock,Combined,1,4.2,42000.0,USD,42000 gallons,USD/gallon,1.0,4.2,2005-10-03,2020-10-28,HU2 starts at 1984-12-03,4,0,0,1,126660.0


In [29]:
# Some indices are the continuation of others
csidata_futures_fact_sheet_df[csidata_futures_fact_sheet_df['SymbolUA'].isin(['YM', 'DJ'])]

,UACsiNumber,SymbolUA,ExchangeSymbol,Exchange,Name,SessionType,IsActive,TerminalPointValue,FullPointValue,Currency,ContractSize,Units,MinimumTick,TickValue,StartDate,EndDate,LinkSymbol,ConversionFactorCode,HasCurrentDayVolume,HasCurrentDayOpenInterest,HasKnownExpirationDates,LastTotalVolume
543,496,DJ,ZD,CBT,Dow Jones Industrial Index,Combined,0,10.0,10.0,USD,$10 x Index,points,1.0,10.0,1997-10-06,2015-06-19,NaN,0,0,0,1,NaN
748,699,YM,YM,CBT,e-mini Dow ($5),Combined,1,5.0,5.0,USD,$5 x Index,points,1.0,5.0,2002-04-05,2020-10-28,DJ starts at 1997-10-06,0,0,0,1,177242.0


## Verify that for all mentioned contracts the files are downloaded and available

In [30]:
# Verify that all the data files are available
ldf['cash_file_available'] = False
ldf['first_future_year'] = ''
ldf['first_future_month'] = ''
ldf['last_future_year'] = ''
ldf['last_future_month'] = ''

for idx, row in ldf_.iterrows():
    csi_symbol = row['csi_symbol']
    csi_file_symbol = csi_symbol.ljust(3, '_')
    dir_path = csi_futures_data.data_path3 + '/' + csi_symbol
    file_name_list = sorted(fnmatch.filter(os.listdir(dir_path), '*.CSV'))
    cash_file_name = file_name_list[0]
    
    r = re.match(r'^{}(\d\d\d\d)(.).*$'.format(csi_file_symbol), cash_file_name)
    if not r:
        raise RuntimeError('cash_file_name does not match file pattern')
    cash_file_available = r.group(2) == '$'
    ldf_.loc[idx, 'cash_file_available'] = cash_file_available
    if cash_file_available:
        futures_file_name_list_start = 1
    else:
        futures_file_name_list_start = 0
    
    futures_file_name_list = file_name_list[futures_file_name_list_start:]
    
    first_future = futures_file_name_list[0]
    r = re.match(r'^{}(\d\d\d\d)(.).*$'.format(csi_file_symbol), first_future)
    if not r:
        raise RuntimeError('first_future does not match file pattern')
    first_future_year  = r.group(1)
    first_future_month = r.group(2)
    
    last_future = futures_file_name_list[-1]
    r = re.match(r'^{}(\d\d\d\d)(.).*$'.format(csi_file_symbol), last_future)
    if not r:
        raise RuntimeError('last_future does not match file pattern')
    last_future_year  = r.group(1)
    last_future_month = r.group(2)
    ldf_.loc[idx, 'first_future_year'] = first_future_year
    ldf_.loc[idx, 'first_future_month'] = first_future_month
    ldf_.loc[idx, 'last_future_year'] = last_future_year
    ldf_.loc[idx, 'last_future_month'] = last_future_month

In [31]:
ldf2 = ldf_[['csi_symbol','root_symbol','multiplier','minor_fx_adj','description','sector', 'ExchangeSymbol','Exchange','Name','SessionType','FullPointValue','Currency','ContractSize','Units','StartDate','EndDate','LastTotalVolume','calc_poin_value',
 'factsheet_minor_fx_adj', 'cash_file_available','first_future_year','first_future_month','last_future_year','last_future_month']]
ldf2

,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,sector,ExchangeSymbol,Exchange,Name,SessionType,FullPointValue,Currency,ContractSize,Units,StartDate,EndDate,LastTotalVolume,calc_poin_value,factsheet_minor_fx_adj,cash_file_available,first_future_year,first_future_month,last_future_year,last_future_month
0,KC2,KC,37500.0,0.01,Coffee,Agricultural,KC,ICEUS,Coffee,Combined,375.0,USD,37500 lbs,cents/pound,1972-08-16,2020-10-28,31867.0,375.0,0.01,True,2000,Z,2023,U
1,C2,_C,5000.0,0.01,Corn,Agricultural,C,CBT,Corn,Combined,50.0,USD,5000 bu,cents/bu,1949-01-03,2020-10-28,426691.0,50.0,0.01,True,2000,X,2023,Z
2,C2,CN,5000.0,0.01,Corn,Agricultural,C,CBT,Corn,Combined,50.0,USD,5000 bu,cents/bu,1949-01-03,2020-10-28,426691.0,50.0,0.01,True,2000,X,2023,Z
3,CT2,CT,50000.0,0.01,Cotton #2,Agricultural,CT,ICEUS,Cotton #2,Combined,500.0,USD,50000 lbs,cents/pound,1967-03-22,2020-10-28,35749.0,500.0,0.01,True,2000,V,2023,N
4,LH,LH,40000.0,0.01,Lean Hogs,Agricultural,LH,CME,Lean Hogs,Combined,400.0,USD,40000 lbs,cents/lb,1966-02-28,2020-10-28,30958.0,400.0,0.01,True,2000,V,2022,J
5,LC,LC,40000.0,0.01,Live Cattle,Agricultural,LC,CME,Live Cattle,Combined,400.0,USD,40000 lbs,cents/lb,1964-11-30,2020-10-28,57781.0,400.0,0.01,True,2000,V,2022,G
6,LB,LB,110.0,1.00,Lumber,Agricultural,LB,CME,Lumber,Combined,110.0,USD,110000 board ft,USD/1000 board ft,1969-10-01,2020-10-28,465.0,110.0,1.00,True,2000,X,2021,X
7,O2,_O,5000.0,0.01,Oats-CBT,Agricultural,O,CBT,Oats,Combined,50.0,USD,5000 bu,cents/bu,1949-01-03,2020-10-28,534.0,50.0,0.01,True,2000,Z,2023,U
8,O2,OA,5000.0,0.01,Oats-CBT,Agricultural,O,CBT,Oats,Combined,50.0,USD,5000 bu,cents/bu,1949-01-03,2020-10-28,534.0,50.0,0.01,True,2000,Z,2023,U
9,OJ2,OJ,15000.0,0.01,Orange Juice,Agricultural,OJ,ICEUS,Frozen Orange Juice,Combined,150.0,USD,15000 lbs,cents/pound,1966-10-26,2020-10-28,768.0,150.0,0.01,False,2000,X,2023,U


In [32]:
# See above: not all contracts started at the beginning of the time window at 2000
# ldf2[ldf2['first_future_year'] > '2000']

In [33]:
# ldf2[ldf2['last_future_year'] < '2020']

In [34]:
# ldf2[ldf2['StartDate'] > pd.Timestamp('2000-12-31')]

In [35]:
#ldf_.to_excel('futures_portfolio_overview.xlsx')

## Get an overview of all contracts plus information about their usage in the different trading strategies described by Andreas

In [36]:
ldf = csi_futures_data.futures_markets_lookup.copy()
ldf['any_market'] = ldf['trend_following_markets'] | ldf['time_return_markets'] | ldf['counter_trend_markets'] | ldf['curve_trading_markets']
ldf.head()

,index,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,is_in_bundle,ROOT_SYMBOL_TO_ETA,FUTURE_EXCHANGE_FEES_BY_SYMBOL,trend_following_markets,time_return_markets,counter_trend_markets,curve_trading_markets,any_market
root_symbol,,,,,,,,,,,,,,,,
KC,0.0,KC2,KC,37500.0,0.01,Coffee,CSCE,agricultural,True,False,False,True,True,True,True,True
_C,1.0,C2,_C,5000.0,0.01,Corn,CBT,agricultural,True,False,False,False,False,False,False,False
CN,2.0,C2,CN,5000.0,0.01,Corn,CBT,agricultural,True,True,True,True,True,False,True,True
CT,3.0,CT2,CT,50000.0,0.01,Cotton #2,NYCE,agricultural,True,False,False,True,True,True,True,True
LH,4.0,LH,LH,40000.0,0.01,Lean Hogs,CME,agricultural,True,True,True,False,False,False,False,False


In [37]:
# ldf.to_excel('futures_markets_lookup.xlsx')

In [38]:
# I checked manually: the following are all not traded any more. They were part of the Quantopian futures offering in 2018.
ldf[~ldf['is_in_bundle']]

,index,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,is_in_bundle,ROOT_SYMBOL_TO_ETA,FUTURE_EXCHANGE_FEES_BY_SYMBOL,trend_following_markets,time_return_markets,counter_trend_markets,curve_trading_markets,any_market
root_symbol,,,,,,,,,,,,,,,,
BD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
DJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
EL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
MD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
ND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
PB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False


In [39]:
# The zipline.finance.constants.ROOT_SYMBOL_TO_ETA and zipline.finance.constants.FUTURE_EXCHANGE_FEES_BY_SYMBOL variables agree with each other
ldf[ldf['ROOT_SYMBOL_TO_ETA'] != ldf['FUTURE_EXCHANGE_FEES_BY_SYMBOL']]

,index,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,is_in_bundle,ROOT_SYMBOL_TO_ETA,FUTURE_EXCHANGE_FEES_BY_SYMBOL,trend_following_markets,time_return_markets,counter_trend_markets,curve_trading_markets,any_market
root_symbol,,,,,,,,,,,,,,,,


In [40]:
# See above: these are the contracts that are not traded any longer
ldf[pd.isnull(ldf['root_symbol'])]

,index,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,is_in_bundle,ROOT_SYMBOL_TO_ETA,FUTURE_EXCHANGE_FEES_BY_SYMBOL,trend_following_markets,time_return_markets,counter_trend_markets,curve_trading_markets,any_market
root_symbol,,,,,,,,,,,,,,,,
BD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
DJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
EL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
MD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
ND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False
PB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,False,False,False


In [41]:
# The following are the contracts that are mentioned in Andreas' books, but were not part of the Quantopian futures offering
ldf[~pd.isnull(ldf['root_symbol']) & ~ldf['ROOT_SYMBOL_TO_ETA']]

,index,csi_symbol,root_symbol,multiplier,minor_fx_adj,description,exchange,sector,is_in_bundle,ROOT_SYMBOL_TO_ETA,FUTURE_EXCHANGE_FEES_BY_SYMBOL,trend_following_markets,time_return_markets,counter_trend_markets,curve_trading_markets,any_market
root_symbol,,,,,,,,,,,,,,,,
KC,0.0,KC2,KC,37500.0,0.01,Coffee,CSCE,agricultural,True,False,False,True,True,True,True,True
_C,1.0,C2,_C,5000.0,0.01,Corn,CBT,agricultural,True,False,False,False,False,False,False,False
CT,3.0,CT2,CT,50000.0,0.01,Cotton #2,NYCE,agricultural,True,False,False,True,True,True,True,True
_O,7.0,O2,_O,5000.0,0.01,Oats-CBT,CBT,agricultural,True,False,False,False,False,False,False,False
OJ,9.0,OJ2,OJ,15000.0,0.01,Orange Juice,NYCE,agricultural,True,False,False,False,False,True,False,True
_S,11.0,S2,_S,5000.0,0.01,Soybean,CBT,agricultural,True,False,False,False,False,False,False,False
LS,13.0,LSU,LS,50.0,1.00,Sugar #5(White),EURONEXT,agricultural,True,False,False,True,True,True,False,True
_W,14.0,W2,_W,5000.0,0.01,Wheat,CBT,agricultural,True,False,False,False,False,False,False,False
KW,16.0,KW2,KW,5000.0,0.01,Wheat-Kansas City,KCBT,agricultural,True,False,False,False,False,False,False,False


In [42]:
# Get all the symbols that Andreas used in his trend_following_markets strategy
tfm1 = csi_futures_data.get_bundle_market_symbols('trend_following_markets')
tfm1

['KC',
 'CN',
 'CT',
 'OA',
 'SY',
 'LS',
 'WC',
 'FC',
 'LR',
 'SM',
 'SB',
 'CL',
 'HO',
 'NG',
 'RB',
 'GC',
 'HG',
 'PA',
 'PL',
 'SV',
 'LG',
 'AD',
 'BP',
 'EC',
 'CD',
 'JY',
 'NZ',
 'SF',
 'DX',
 'ES',
 'NQ',
 'VX',
 'YM',
 'NK',
 'TW',
 'FV',
 'ED',
 'US',
 'TU',
 'TY']

In [43]:
tfm2 = csi_futures_data.trend_following_markets
tfm2

['AD',
 'BP',
 'CD',
 'EC',
 'DX',
 'JY',
 'NZ',
 'SF',
 'CN',
 'CT',
 'FC',
 'KC',
 'LR',
 'LS',
 'OA',
 'SY',
 'SB',
 'SM',
 'WC',
 'BL',
 'CL',
 'GC',
 'HG',
 'HO',
 'LG',
 'NG',
 'PA',
 'PL',
 'RB',
 'SV',
 'ES',
 'NK',
 'NQ',
 'TW',
 'VX',
 'YM',
 'ED',
 'FV',
 'TU',
 'TY',
 'US']

In [44]:
set(tfm1) - set(tfm2)

set()

In [45]:
set(tfm2) - set(tfm1)

{'BL'}